In [1]:
import numpy as np
from itertools import product

In [2]:
radius   = 1737400
alt      = 2000000
ground   = 7.5
exposure = 0.005
samples  = 1000
lines    = 1000

In [3]:
sensor_rad = radius + alt
angle_per_line = ground / radius
angle_per_samp = angle_per_line
angle_per_Second = angle_per_line / exposure

In [4]:
line_vec = np.arange(0, lines+0.00000001)
sample_vec = np.arange(0, samples+0.00000001)
# From here on, matrix indexing is [line, sample, (xyz)]
sample_mat, line_mat = np.meshgrid(line_vec, sample_vec)

In [5]:
positions = sensor_rad * np.vstack([np.cos(-angle_per_line * line_vec), np.zeros(line_vec.shape), np.sin(-angle_per_line * line_vec)]).T
# Note: The chain rule results in an extra negative on the velocity calculations
velocities = sensor_rad * np.vstack([np.sin(-angle_per_line * line_vec), np.zeros(line_vec.shape), -np.cos(-angle_per_line * line_vec)]).T
print('Positions')
for pos in positions[::int(0.25/exposure)]:
    print(list(pos))
print('Velocities')
for vel in velocities[::int(0.25/exposure)]:
    print(list(vel))

Positions
[3737400.0, 0.0, -0.0]
[3737399.912943243, 0.0, -806.6795148600813]
[3737399.651772976, 0.0, -1613.3589921395437]
[3737399.216489211, 0.0, -2420.03839425777]
[3737398.607091969, 0.0, -3226.7176836341473]
[3737397.823581278, 0.0, -4033.396822688066]
[3737396.8659571735, 0.0, -4840.075773838925]
[3737395.734219702, 0.0, -5646.754499506133]
[3737394.4283689144, 0.0, -6453.432962109106]
[3737392.9484048723, 0.0, -7260.111124067275]
[3737391.2943276456, 0.0, -8066.788947800085]
[3737389.4661373096, 0.0, -8873.466395726995]
[3737387.4638339505, 0.0, -9680.14343026748]
[3737385.287417662, 0.0, -10486.820013841041]
[3737382.936888544, 0.0, -11293.496108867193]
[3737380.4122467074, 0.0, -12100.17167776548]
[3737377.713492269, 0.0, -12906.846682955462]
[3737374.840625355, 0.0, -13713.521086856732]
[3737371.7936460995, 0.0, -14520.194851888911]
[3737368.5725546437, 0.0, -15326.867940471646]
[3737365.177351138, 0.0, -16133.540315024618]
Velocities
[-0.0, 0.0, -3737400.0]
[-806.6795148600

In [6]:
lat = -angle_per_line * line_mat
# Image is a right look, so the longitude goes negative
lon = -angle_per_samp * sample_mat
ground_points = radius * np.stack([np.multiply(np.cos(lat), np.cos(lon)), np.multiply(np.cos(lat), np.sin(lon)), np.sin(lat)], axis=-1)
print("Ground point at line: 500, sample: 500")
print(ground_points[500, 500])

Ground point at line: 500, sample: 500
[1737391.90602155   -3749.98835331   -3749.99708833]


In [7]:
slant_range = np.array([[np.linalg.norm(point) for point in row] for row in ground_points - positions[:, None, :]])
ground_range = radius * np.abs(lon)

In [8]:
range_poly = np.polynomial.polynomial.polyfit(ground_range.flatten(), slant_range.flatten(), 3)
print("Ground range to slant range polynomial coefficients")
print(list(range_poly))

Ground range to slant range polynomial coefficients
[2000000.000003915, -1.0488420462327845e-08, 5.377893056639776e-07, -1.3072058387193456e-15]


In [9]:
sc_pos = positions[500]
sc_vel = velocities[500]
off_ground_pt = ground_points[500, 500] - np.array([100, 100, 100])
look_vec = off_ground_pt - positions[500]
zero_doppler_look_vec = look_vec - np.dot(look_vec, sc_vel) / np.dot(sc_vel, sc_vel) * sc_vel
locus_point = sc_pos + slant_range[500, 500] / np.linalg.norm(zero_doppler_look_vec) * zero_doppler_look_vec
# Image is a right look, so do look X velocity
locus_direction = np.cross(zero_doppler_look_vec, sc_vel)
locus_direction = 1.0 / np.linalg.norm(locus_direction) * locus_direction
print("Locus point:", list(locus_point))
print("Locus direction:", list(locus_direction))

Locus point: [1737392.0956685692, -3849.7959147875467, -3749.9887626446034]
Locus direction: [0.0019248861951120758, -0.9999981473962212, -4.154676206387554e-06]


In [10]:
remote_look_vec = -slant_range[500, 500] / sensor_rad * sc_pos
remote_zero_doppler_look_vec = remote_look_vec - np.dot(remote_look_vec, sc_vel) / np.dot(sc_vel, sc_vel) * sc_vel
remote_locus_point = sc_pos + remote_zero_doppler_look_vec
remote_locus_direction = np.cross(remote_zero_doppler_look_vec, sc_vel)
remote_locus_direction = 1.0 / np.linalg.norm(remote_locus_direction) * remote_locus_direction
print("Remote locus point:", list(remote_locus_point))
print("Remote locus direction:", list(remote_locus_direction))

Remote locus point: [1737388.3904556318, 0.0, -3749.980765309453]
Remote locus direction: [-0.0, -1.0, 0.0]
